In [13]:
import sys
sys.path.append('../')
import re
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import datetime

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
matplotlib.style.use('ggplot')

In [14]:
train_df = pd.read_csv('../input/original/train_round_0.csv', sep=',')
test_df = pd.read_csv('../input/original/test_data_r0.csv', sep=',')

In [15]:
# 读取预测结果
test_pred = pd.read_csv('add_back_trans_combine_0315pred-entity_0928.csv')

test_pred = pd.merge(test_pred, test_df[['question_id', 'question']], on='question_id')

In [16]:
test_pred.head()

,question_id,answer,question
0,Q_32174nsdcj,AS-12型核潜艇,世界上最小的核潜艇是哪艘？
1,Q_42346jgcgs,大口径火炮,第二次世界大战时期，对于水面战斗舰艇来讲，最具威力的武器是什么？
2,Q_01814skdqn,拉美国际军事合作部门,中国国防部于6月12日至27日在国防大学国际防务学院举办什么合作部门主官研讨班?
3,Q_12633dcgkq,F-15战机,12月13日，中国海监飞机和舰船对钓鱼岛开展首次海空立体巡航时日本紧急出动航空自卫队8架什么...
4,Q_01789dggqk,4枚,作为俄罗斯空基核打击力量的重要补充，而且还可以作为新的力量投射战略的先锋的战机可以挂载多少枚...


In [17]:
import re

ans_pattern = re.compile(r'@content\d@')
def find_answer_in_docid(answer):
    docs = ans_pattern.findall(answer)
    return list(set([int(doc[-2:-1]) for doc in docs]))

def get_train_sample_answer_str(answer):
    answer_texts = []
    
    ans_in_docids = find_answer_in_docid(answer)
    for ans_in_docid in ans_in_docids:
        answer_strs = answer.split('@content{}@'.format(ans_in_docid))
        for answer_str in answer_strs:
            answer_str = answer_str.strip()
            if answer_str != '' and '@content' not in answer_str:
                answer_str = answer_str.replace('content{}@'.format(ans_in_docid), '')
                answer_texts.append(answer_str)
    
    if len(answer_texts) > 1:
        return '#'.join(answer_texts)
    else:
        return answer_texts[0]
    

In [18]:
train_ques_answer = {}

for idx, row in train_df.iterrows():
    train_ques_answer[row['question']] = get_train_sample_answer_str(row['answer'])

In [19]:
len(train_ques_answer)

24792

In [20]:
def replace_answer(row):
    if row['question'] in train_ques_answer:
        print('pred: {}, train_answer: {}'.format(row['answer'], train_ques_answer[row['question']]))
        return train_ques_answer[row['question']]
    else:
        return row['answer']

In [21]:
test_pred['answer'] = test_pred.apply(lambda row: replace_answer(row), axis=1)

pred: 大口径火炮, train_answer: 大口径火炮
pred: 高航速、远航程、大深度、大威力、智能化、隐身, train_answer: 现代鱼雷具有速度快、航程远、隐蔽性好、命中率高和破坏威力大等特点，被称“水中导弹”。
pred: 每小时70公里, train_answer: 每小时70公里
pred: 南部九四式手枪全长180毫米, train_answer: 南部九四式手枪全长 180毫米
pred: 19.6公斤, train_answer: 19.6公斤
pred: “果断风暴”, train_answer: “果断风暴”
pred: 09式霰弹枪, train_answer: 09式霰弹枪
pred: R-77, train_answer: R-77
pred: 最大特征就是大威力的120毫米反坦克炮。, train_answer: 最大特征就是大威力的120毫米反坦克炮。
pred: 整齐划一、铿锵有力、威武雄壮”, train_answer: 整齐划一、铿锵有力、威武雄壮
pred: 2016年, train_answer: 2016年
pred: 4218千克, train_answer: 只有4.22吨
pred: 超过了22公里, train_answer: 超过了22公里
pred: 日本“摩耶”号导弹驱逐舰, train_answer: 日本“摩耶”号导弹驱逐舰
pred: “杰里科”导弹、“杰里科-2”近程导弹和“杰里科-3”型远程弹道导弹, train_answer: 分为“杰里科”导弹、“杰里科-2”近程导弹和“杰里科-3”型远程弹道导弹三种型号。
pred: T-34, train_answer: T-34
pred: 250千米, train_answer: 250千米
pred: 5.8m, train_answer: 5.8m
pred: 前甲板的MK38型25mm舰炮, train_answer: MK38型25mm舰炮
pred: “红宝石”级全长73.6米, train_answer: 73.6米
pred: 436千克, train_answer: 436千克


In [22]:
del test_pred['question']

In [23]:
test_pred.shape

(4969, 2)

In [24]:
test_pred.to_csv('add_back_trans_combine_0315pred-entity_0928_postprocess.csv', index=False, encoding='utf8')